# 🚀 Gray-Scott Phase 4: Last 64 Frames GPU版（Google Colab）

**目標**: 後半64フレームのみを使用した学習でPhase 1 (0.565) → Phase 4 (0.65+) へのさらなる向上

**主要改善点**:
- ✅ 残差接続（ResNet）+ 時空間注意機構
- ✅ GPU高速化（CPU比 5-10倍）
- ✅ Google Drive連携
- ✅ **後半64フレームのみを使用** - より安定した後期パターンに焦点

**前提条件**: Google Driveの`マイドライブ/GrayScottML/gif/`に1500個のGIFファイルを配置

**実行時間**: **GPU 3-5分** 🏃‍♂️💨

## 更新履歴
- 2025-08-11: Data augmentation（平行移動・90/180/270度回転・水平/垂直反転）を導入。
  - 学習: GrayScottDatasetLast64Aug(augment=True) でオンザフライ適用
  - 評価/潜在ベクトル抽出: augment=False でデータ分布を固定


## 📋 Step 1: 環境セットアップ & Google Drive接続

In [ ]:
# Quick run mode: use only first N GIF files
QUICK_RUN = True
max_samples = 20 if QUICK_RUN else None
print('Quick run max_samples =', max_samples)


In [ ]:
# Embedded Phase 4 model code (no external src import needed)
#!/usr/bin/env python3
"""
Gray-Scott 3D CNN Autoencoder - Phase 4: 対比学習・評価改善
目標: Phase 3 (0.5144) → Phase 4 (0.55+) への更なる向上

主要改善点:
- 対比学習（Contrastive Learning）
- 階層的クラスタリング分析
- 包括的評価指標
- 改善された訓練ループ
"""

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import imageio.v2 as imageio
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import time
import pickle
from typing import List, Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# GPU設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Phase 4 Using device: {device}")

# ================================
# 1. 対比学習システム
# ================================

class ContrastiveLoss(nn.Module):
    """f-kパラメータ類似性に基づく対比学習損失"""
    
    def __init__(self, temperature=0.5, margin=1.0):
        super().__init__()
        self.temperature = temperature
        self.margin = margin
        self.cosine_similarity = nn.CosineSimilarity(dim=2)
    
    def forward(self, features, f_params, k_params):
        """
        Args:
            features: (batch_size, feature_dim) - 潜在表現
            f_params: (batch_size,) - fパラメータ
            k_params: (batch_size,) - kパラメータ
        """
        batch_size = features.size(0)
        
        # f-kパラメータ空間での類似性計算
        f_diff = torch.abs(f_params.unsqueeze(1) - f_params.unsqueeze(0))
        k_diff = torch.abs(k_params.unsqueeze(1) - k_params.unsqueeze(0))
        
        # 正規化されたパラメータ距離
        param_distance = torch.sqrt(f_diff**2 + k_diff**2)
        
        # 類似性閾値（近いパラメータは類似、遠いパラメータは非類似）
        similarity_threshold = 0.01  # f-k空間での閾値
        positive_mask = param_distance < similarity_threshold
        negative_mask = param_distance > similarity_threshold * 3
        
        # 特徴量の類似性計算
        features_norm = F.normalize(features, p=2, dim=1)
        similarity_matrix = torch.mm(features_norm, features_norm.t()) / self.temperature
        
        # 対比学習損失
        positive_loss = 0
        negative_loss = 0
        
        if positive_mask.sum() > 0:
            positive_sim = similarity_matrix[positive_mask]
            positive_loss = -torch.log(torch.exp(positive_sim).sum() / torch.exp(similarity_matrix).sum())
        
        if negative_mask.sum() > 0:
            negative_sim = similarity_matrix[negative_mask]
            negative_loss = torch.log(torch.exp(negative_sim).sum() / torch.exp(similarity_matrix).sum())
        
        contrastive_loss = positive_loss + negative_loss
        
        return contrastive_loss

class ProjectionHead(nn.Module):
    """対比学習用射影ヘッド"""
    
    def __init__(self, input_dim=512, hidden_dim=256, output_dim=128):
        super().__init__()
        self.projection = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, output_dim),
            nn.BatchNorm1d(output_dim)
        )
    
    def forward(self, x):
        return self.projection(x)

# ================================
# 2. 階層的クラスタリング分析
# ================================

class HierarchicalClusteringAnalysis:
    """階層的クラスタリング分析システム"""
    
    def __init__(self, method='ward', metric='euclidean'):
        self.method = method
        self.metric = metric
        self.linkage_matrix = None
        self.optimal_clusters = None
    
    def fit(self, features):
        """階層的クラスタリング実行"""
        # 特徴量の標準化
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
        
        # 階層的クラスタリング
        self.linkage_matrix = linkage(features_scaled, method=self.method, metric=self.metric)
        
        # 最適クラスタ数の決定
        self.optimal_clusters = self._find_optimal_clusters(features_scaled)
        
        return self
    
    def _find_optimal_clusters(self, features, max_clusters=20):
        """最適クラスタ数の自動決定"""
        silhouette_scores = []
        cluster_range = range(2, min(max_clusters + 1, len(features) // 2))
        
        for n_clusters in cluster_range:
            cluster_labels = fcluster(self.linkage_matrix, n_clusters, criterion='maxclust')
            
            if len(np.unique(cluster_labels)) > 1:
                silhouette_avg = silhouette_score(features, cluster_labels)
                silhouette_scores.append(silhouette_avg)
            else:
                silhouette_scores.append(-1)
        
        if silhouette_scores:
            optimal_idx = np.argmax(silhouette_scores)
            optimal_n_clusters = cluster_range[optimal_idx]
            return optimal_n_clusters
        else:
            return 2
    
    def get_cluster_labels(self, n_clusters=None):
        """クラスタラベルの取得"""
        if n_clusters is None:
            n_clusters = self.optimal_clusters
        
        return fcluster(self.linkage_matrix, n_clusters, criterion='maxclust')
    
    def plot_dendrogram(self, figsize=(12, 8)):
        """デンドログラム可視化"""
        plt.figure(figsize=figsize)
        dendrogram(self.linkage_matrix, truncate_mode='level', p=10)
        plt.title('Hierarchical Clustering Dendrogram')
        plt.xlabel('Sample Index')
        plt.ylabel('Distance')
        plt.show()

# ================================
# 3. 包括的評価指標
# ================================

class ComprehensiveEvaluationMetrics:
    """包括的評価指標システム"""
    
    def __init__(self):
        self.metrics = {}
    
    def calculate_all_metrics(self, features, labels, f_params=None, k_params=None):
        """全ての評価指標を計算"""
        
        # 基本クラスタリング指標
        self.metrics['silhouette_score'] = silhouette_score(features, labels)
        self.metrics['calinski_harabasz_score'] = calinski_harabasz_score(features, labels)
        self.metrics['davies_bouldin_score'] = davies_bouldin_score(features, labels)
        
        # 近傍一致度指標
        self.metrics['neighborhood_agreement'] = self._calculate_neighborhood_agreement(features, labels)
        
        # パラメータ空間分離度評価
        if f_params is not None and k_params is not None:
            self.metrics['parameter_separation'] = self._calculate_parameter_separation(
                features, labels, f_params, k_params
            )
        
        # クラスタ内分散・クラスタ間分散
        self.metrics['within_cluster_variance'] = self._calculate_within_cluster_variance(features, labels)
        self.metrics['between_cluster_variance'] = self._calculate_between_cluster_variance(features, labels)
        
        # 安定性指標
        self.metrics['cluster_stability'] = self._calculate_cluster_stability(features, labels)
        
        return self.metrics
    
    def _calculate_neighborhood_agreement(self, features, labels, k=10):
        """近傍一致度の計算"""
        from sklearn.neighbors import NearestNeighbors
        
        nbrs = NearestNeighbors(n_neighbors=k+1).fit(features)
        distances, indices = nbrs.kneighbors(features)
        
        agreements = []
        for i in range(len(features)):
            neighbor_labels = labels[indices[i][1:]]  # 自分以外の近傍
            same_cluster = np.sum(neighbor_labels == labels[i])
            agreement = same_cluster / k
            agreements.append(agreement)
        
        return np.mean(agreements)
    
    def _calculate_parameter_separation(self, features, labels, f_params, k_params):
        """パラメータ空間分離度の計算"""
        unique_labels = np.unique(labels)
        separations = []
        
        for label in unique_labels:
            mask = labels == label
            if np.sum(mask) > 1:
                cluster_f = f_params[mask]
                cluster_k = k_params[mask]
                
                # クラスタ内のパラメータ分散
                f_var = np.var(cluster_f)
                k_var = np.var(cluster_k)
                cluster_variance = f_var + k_var
                
                separations.append(cluster_variance)
        
        return np.mean(separations) if separations else 0
    
    def _calculate_within_cluster_variance(self, features, labels):
        """クラスタ内分散の計算"""
        unique_labels = np.unique(labels)
        within_variances = []
        
        for label in unique_labels:
            mask = labels == label
            if np.sum(mask) > 1:
                cluster_features = features[mask]
                centroid = np.mean(cluster_features, axis=0)
                variance = np.mean(np.sum((cluster_features - centroid)**2, axis=1))
                within_variances.append(variance)
        
        return np.mean(within_variances) if within_variances else 0
    
    def _calculate_between_cluster_variance(self, features, labels):
        """クラスタ間分散の計算"""
        unique_labels = np.unique(labels)
        centroids = []
        
        for label in unique_labels:
            mask = labels == label
            centroid = np.mean(features[mask], axis=0)
            centroids.append(centroid)
        
        centroids = np.array(centroids)
        overall_centroid = np.mean(centroids, axis=0)
        
        between_variance = np.mean(np.sum((centroids - overall_centroid)**2, axis=1))
        return between_variance
    
    def _calculate_cluster_stability(self, features, labels, n_bootstrap=10):
        """クラスタ安定性の計算"""
        from sklearn.utils import resample
        
        original_labels = labels
        stability_scores = []
        
        for _ in range(n_bootstrap):
            # ブートストラップサンプリング
            bootstrap_indices = resample(range(len(features)), n_samples=len(features))
            bootstrap_features = features[bootstrap_indices]
            bootstrap_labels = labels[bootstrap_indices]
            
            # クラスタリング実行
            kmeans = KMeans(n_clusters=len(np.unique(original_labels)), random_state=42)
            new_labels = kmeans.fit_predict(bootstrap_features)
            
            # ラベル一致度計算（ハンガリアンアルゴリズム簡易版）
            agreement = self._calculate_label_agreement(bootstrap_labels, new_labels)
            stability_scores.append(agreement)
        
        return np.mean(stability_scores)
    
    def _calculate_label_agreement(self, labels1, labels2):
        """ラベル一致度の計算"""
        from scipy.optimize import linear_sum_assignment
        
        unique_labels1 = np.unique(labels1)
        unique_labels2 = np.unique(labels2)
        
        # 混同行列作成
        confusion_matrix = np.zeros((len(unique_labels1), len(unique_labels2)))
        
        for i, label1 in enumerate(unique_labels1):
            for j, label2 in enumerate(unique_labels2):
                confusion_matrix[i, j] = np.sum((labels1 == label1) & (labels2 == label2))
        
        # ハンガリアンアルゴリズムで最適割り当て
        row_ind, col_ind = linear_sum_assignment(-confusion_matrix)
        
        # 一致度計算
        agreement = confusion_matrix[row_ind, col_ind].sum() / len(labels1)
        return agreement
    
    def print_metrics(self):
        """評価指標の表示"""
        print("\n🎯 Phase 4 包括的評価指標")
        print("=" * 50)
        
        # 基本指標
        print(f"Silhouette Score: {self.metrics.get('silhouette_score', 0):.4f}")
        print(f"Calinski-Harabasz: {self.metrics.get('calinski_harabasz_score', 0):.2f}")
        print(f"Davies-Bouldin: {self.metrics.get('davies_bouldin_score', 0):.4f}")
        
        # 高度な指標
        print(f"Neighborhood Agreement: {self.metrics.get('neighborhood_agreement', 0):.4f}")
        print(f"Parameter Separation: {self.metrics.get('parameter_separation', 0):.4f}")
        print(f"Within Cluster Variance: {self.metrics.get('within_cluster_variance', 0):.4f}")
        print(f"Between Cluster Variance: {self.metrics.get('between_cluster_variance', 0):.4f}")
        print(f"Cluster Stability: {self.metrics.get('cluster_stability', 0):.4f}")

# ================================
# 4. Phase 3ベースのモデル拡張
# ================================

class GrayScottAugmentation:
    """Gray-Scott専用データ拡張クラス（Phase 3から継承）"""
    
    def __init__(self, 
                 temporal_shift_prob=0.3,
                 spatial_flip_prob=0.5,
                 noise_prob=0.2,
                 intensity_prob=0.3,
                 temporal_crop_prob=0.2):
        self.temporal_shift_prob = temporal_shift_prob
        self.spatial_flip_prob = spatial_flip_prob
        self.noise_prob = noise_prob
        self.intensity_prob = intensity_prob
        self.temporal_crop_prob = temporal_crop_prob
    
    def temporal_shift(self, tensor, max_shift=3):
        """時間軸シフト"""
        if np.random.random() < self.temporal_shift_prob:
            shift = np.random.randint(-max_shift, max_shift + 1)
            if shift != 0:
                tensor = torch.roll(tensor, shift, dims=1)
        return tensor
    
    def spatial_flip(self, tensor):
        """空間軸反転"""
        if np.random.random() < self.spatial_flip_prob:
            if np.random.random() < 0.5:
                tensor = torch.flip(tensor, dims=[3])
            if np.random.random() < 0.5:
                tensor = torch.flip(tensor, dims=[2])
        return tensor
    
    def add_noise(self, tensor, noise_std=0.02):
        """ガウシアンノイズ追加"""
        if np.random.random() < self.noise_prob:
            noise = torch.randn_like(tensor) * noise_std
            tensor = torch.clamp(tensor + noise, 0, 1)
        return tensor
    
    def intensity_transform(self, tensor, gamma_range=(0.8, 1.2)):
        """強度変換"""
        if np.random.random() < self.intensity_prob:
            gamma = np.random.uniform(*gamma_range)
            tensor = torch.pow(tensor, gamma)
        return tensor
    
    def temporal_crop(self, tensor, crop_ratio=0.1):
        """時間軸クロップ"""
        if np.random.random() < self.temporal_crop_prob:
            T = tensor.shape[1]
            crop_size = int(T * crop_ratio)
            start_idx = np.random.randint(0, crop_size + 1)
            end_idx = T - np.random.randint(0, crop_size + 1)
            
            cropped = tensor[:, start_idx:end_idx]
            tensor = F.interpolate(cropped.unsqueeze(0), size=(T, tensor.shape[2], tensor.shape[3]), 
                                 mode='trilinear', align_corners=False).squeeze(0)
        return tensor
    
    def __call__(self, tensor):
        """全ての拡張を適用"""
        tensor = self.temporal_shift(tensor)
        tensor = self.spatial_flip(tensor)
        tensor = self.add_noise(tensor)
        tensor = self.intensity_transform(tensor)
        tensor = self.temporal_crop(tensor)
        return tensor

class MultiScaleFeatureFusion(nn.Module):
    """マルチスケール特徴融合（Phase 3から継承）"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        # 4つの異なるスケールでの特徴抽出
        self.scale1 = nn.Conv3d(in_channels, out_channels // 4, kernel_size=1, padding=0)  # Point-wise
        self.scale2 = nn.Conv3d(in_channels, out_channels // 4, kernel_size=3, padding=1)  # Local
        self.scale3 = nn.Conv3d(in_channels, out_channels // 4, kernel_size=5, padding=2)  # Global
        
        # プーリングベースの特徴
        self.pool = nn.AdaptiveAvgPool3d(1)
        self.scale4 = nn.Conv3d(in_channels, out_channels // 4, kernel_size=1)
        
        self.bn = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        # 各スケールで特徴抽出
        feat1 = self.scale1(x)
        feat2 = self.scale2(x)
        feat3 = self.scale3(x)
        
        # プーリング特徴
        pooled = self.pool(x)
        feat4 = self.scale4(pooled)
        feat4 = feat4.expand_as(feat1)
        
        # 特徴融合
        fused = torch.cat([feat1, feat2, feat3, feat4], dim=1)
        fused = self.bn(fused)
        fused = self.relu(fused)
        
        return fused

class EnhancedSpatioTemporalAttention(nn.Module):
    """改良時空間注意機構（Phase 3から継承）"""
    
    def __init__(self, in_channels, reduction=16):
        super().__init__()
        
        # 時間注意
        self.temporal_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d((None, 1, 1)),
            nn.Conv3d(in_channels, in_channels // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(in_channels // reduction, in_channels, kernel_size=1),
            nn.Sigmoid()
        )
        
        # 空間注意
        self.spatial_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d((1, None, None)),
            nn.Conv3d(in_channels, in_channels // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(in_channels // reduction, in_channels, kernel_size=1),
            nn.Sigmoid()
        )
        
        # チャネル注意
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d(1),
            nn.Conv3d(in_channels, in_channels // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(in_channels // reduction, in_channels, kernel_size=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # 各注意機構を適用
        temp_att = self.temporal_attention(x)
        spat_att = self.spatial_attention(x)
        chan_att = self.channel_attention(x)
        
        # 注意重みを適用
        x = x * temp_att * spat_att * chan_att
        
        return x

class ResidualMultiScaleBlock3D(nn.Module):
    """残差マルチスケールブロック（Phase 3から継承）"""
    
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        self.multi_scale = MultiScaleFeatureFusion(in_channels, out_channels)
        self.attention = EnhancedSpatioTemporalAttention(out_channels)
        
        # 残差接続用
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(out_channels)
            )
    
    def forward(self, x):
        residual = self.shortcut(x)
        
        out = self.multi_scale(x)
        out = self.attention(out)
        
        out += residual
        return out

class Conv3DAutoencoderPhase4(nn.Module):
    """Phase 4: 対比学習統合オートエンコーダー"""
    
    def __init__(self, latent_dim=512, input_shape=(20, 64, 64)):
        super().__init__()
        
        self.latent_dim = latent_dim
        self.input_shape = input_shape
        
        # エンコーダー
        self.encoder = nn.Sequential(
            # 入力: (1, 20, 64, 64)
            ResidualMultiScaleBlock3D(1, 32),
            nn.MaxPool3d(2),  # (32, 10, 32, 32)
            
            ResidualMultiScaleBlock3D(32, 64),
            nn.MaxPool3d(2),  # (64, 5, 16, 16)
            
            ResidualMultiScaleBlock3D(64, 128),
            nn.MaxPool3d(2),  # (128, 2, 8, 8)
            
            ResidualMultiScaleBlock3D(128, 256),
            nn.AdaptiveAvgPool3d(1),  # (256, 1, 1, 1)
        )
        
        # 潜在空間
        self.fc_encoder = nn.Sequential(
            nn.Linear(256, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        # 対比学習用射影ヘッド
        self.projection_head = ProjectionHead(latent_dim, 256, 128)
        
        # デコーダー
        self.fc_decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        # デコーダーを個別のレイヤーとして定義（サイズ調整付き）
        self.decoder_conv1 = nn.ConvTranspose3d(256, 128, kernel_size=(3, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
        self.decoder_bn1 = nn.BatchNorm3d(128)
        
        self.decoder_conv2 = nn.ConvTranspose3d(128, 64, kernel_size=(3, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
        self.decoder_bn2 = nn.BatchNorm3d(64)
        
        self.decoder_conv3 = nn.ConvTranspose3d(64, 32, kernel_size=(3, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
        self.decoder_bn3 = nn.BatchNorm3d(32)
        
        self.decoder_conv4 = nn.ConvTranspose3d(32, 1, kernel_size=(3, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
        
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()
    
    def encode(self, x):
        """エンコード"""
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        latent = self.fc_encoder(x)
        return latent
    
    def decode(self, latent):
        """デコード（サイズ調整付き）"""
        x = self.fc_decoder(latent)
        x = x.view(x.size(0), 256, 1, 1, 1)
        
        # 段階的にデコード（各ステップでサイズを調整）
        # (256, 1, 1, 1) -> (128, 2, 8, 8)
        x = self.decoder_conv1(x)
        x = self.decoder_bn1(x)
        x = self.relu(x)
        # サイズ調整
        x = F.interpolate(x, size=(2, 8, 8), mode='trilinear', align_corners=False)
        
        # (128, 2, 8, 8) -> (64, 5, 16, 16)
        x = self.decoder_conv2(x)
        x = self.decoder_bn2(x)
        x = self.relu(x)
        # サイズ調整
        x = F.interpolate(x, size=(5, 16, 16), mode='trilinear', align_corners=False)
        
        # (64, 5, 16, 16) -> (32, 10, 32, 32)
        x = self.decoder_conv3(x)
        x = self.decoder_bn3(x)
        x = self.relu(x)
        # サイズ調整
        x = F.interpolate(x, size=(10, 32, 32), mode='trilinear', align_corners=False)
        
        # (32, 10, 32, 32) -> (1, 20, 64, 64)
        x = self.decoder_conv4(x)
        # 最終サイズ調整
        x = F.interpolate(x, size=(20, 64, 64), mode='trilinear', align_corners=False)
        x = self.sigmoid(x)
        
        return x
    
    def forward(self, x):
        """フォワードパス"""
        latent = self.encode(x)
        reconstructed = self.decode(latent)
        projection = self.projection_head(latent)
        
        return reconstructed, latent, projection

# ================================
# 5. データセットクラス
# ================================

class GrayScottDataset(Dataset):
    """Gray-Scott データセット（Phase 3から継承・拡張）"""
    
    def __init__(self, gif_folder, augmentation=None, max_samples=None):
        self.gif_folder = gif_folder
        self.augmentation = augmentation
        
        # GIFファイルリスト取得
        self.gif_files = [f for f in os.listdir(gif_folder) if f.endswith('.gif')]
        
        if max_samples:
            self.gif_files = self.gif_files[:max_samples]
        
        # f-kパラメータ抽出
        self.f_params = []
        self.k_params = []
        
        for gif_file in self.gif_files:
            f_val, k_val = self.extract_parameters(gif_file)
            self.f_params.append(f_val)
            self.k_params.append(k_val)
        
        self.f_params = np.array(self.f_params)
        self.k_params = np.array(self.k_params)
        
        print(f"📊 Dataset loaded: {len(self.gif_files)} samples")
        print(f"f range: {self.f_params.min():.4f} - {self.f_params.max():.4f}")
        print(f"k range: {self.k_params.min():.4f} - {self.k_params.max():.4f}")
    
    def extract_parameters(self, filename):
        """ファイル名からf-kパラメータを抽出"""
        pattern = r'f([\d.]+)-k([\d.]+)'
        match = re.search(pattern, filename)
        
        if match:
            f_val = float(match.group(1))
            k_val = float(match.group(2))
            return f_val, k_val
        else:
            return 0.0, 0.0
    
    def __len__(self):
        return len(self.gif_files)
    
    def __getitem__(self, idx):
        gif_path = os.path.join(self.gif_folder, self.gif_files[idx])
        
        # GIF読み込み
        gif = imageio.mimread(gif_path)
        
        # 最初の20フレームを取得
        frames = gif[:20] if len(gif) >= 20 else gif
        
        # テンソルに変換
        tensor = torch.FloatTensor(frames).unsqueeze(0)  # (1, T, H, W)
        tensor = tensor / 255.0  # 正規化
        
        # データ拡張
        if self.augmentation:
            tensor = self.augmentation(tensor)
        
        return tensor, self.f_params[idx], self.k_params[idx], idx

# ================================
# 6. 改善された訓練ループ
# ================================

def train_phase4_model(model, dataloader, num_epochs=30, learning_rate=1e-4):
    """Phase 4モデルの訓練"""
    
    # 最適化器
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    # 損失関数
    reconstruction_loss = nn.MSELoss()
    contrastive_loss = ContrastiveLoss(temperature=0.5)
    
    # 訓練ループ
    model.train()
    train_losses = []
    
    print("🚀 Phase 4 Training Started")
    print("=" * 50)
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        epoch_recon_loss = 0.0
        epoch_contrastive_loss = 0.0
        
        for batch_idx, (data, f_params, k_params, _) in enumerate(dataloader):
            data = data.to(device)
            f_params = f_params.to(device)
            k_params = k_params.to(device)
            
            optimizer.zero_grad()
            
            # フォワードパス
            reconstructed, latent, projection = model(data)
            
            # 損失計算
            recon_loss = reconstruction_loss(reconstructed, data)
            contrast_loss = contrastive_loss(projection, f_params, k_params)
            
            # 総損失（重み付き）
            total_loss = recon_loss + 0.1 * contrast_loss
            
            # バックプロパゲーション
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_loss += total_loss.item()
            epoch_recon_loss += recon_loss.item()
            epoch_contrastive_loss += contrast_loss.item()
        
        scheduler.step()
        
        avg_loss = epoch_loss / len(dataloader)
        avg_recon_loss = epoch_recon_loss / len(dataloader)
        avg_contrast_loss = epoch_contrastive_loss / len(dataloader)
        
        train_losses.append(avg_loss)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}]")
            print(f"  Total Loss: {avg_loss:.6f}")
            print(f"  Reconstruction: {avg_recon_loss:.6f}")
            print(f"  Contrastive: {avg_contrast_loss:.6f}")
            print(f"  LR: {scheduler.get_last_lr()[0]:.6f}")
    
    print("✅ Phase 4 Training Completed!")
    return train_losses

# ================================
# 7. メイン実行関数
# ================================

def main():
    """Phase 4メイン実行"""
    
    # データフォルダ設定
    GIF_FOLDER = "path/to/gif/folder"  # 実際のパスに変更
    
    if not os.path.exists(GIF_FOLDER):
        print(f"❌ GIF folder not found: {GIF_FOLDER}")
        return
    
    # データ拡張設定
    augmentation = GrayScottAugmentation()
    
    # データセット作成
    dataset = GrayScottDataset(GIF_FOLDER, augmentation=augmentation)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)
    
    # モデル作成
    model = Conv3DAutoencoderPhase4(latent_dim=512).to(device)
    
    print(f"📊 Model Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # 訓練実行
    train_losses = train_phase4_model(model, dataloader, num_epochs=30)
    
    # モデル保存
    torch.save(model.state_dict(), 'models/phase4_model.pth')
    print("💾 Model saved to models/phase4_model.pth")
    
    # 評価実行
    evaluate_phase4_model(model, dataloader)

def evaluate_phase4_model(model, dataloader):
    """Phase 4モデルの評価"""
    
    model.eval()
    all_latents = []
    all_f_params = []
    all_k_params = []
    
    print("🔍 Phase 4 Evaluation Started")
    
    with torch.no_grad():
        for data, f_params, k_params, _ in dataloader:
            data = data.to(device)
            _, latent, _ = model(data)
            
            all_latents.append(latent.cpu().numpy())
            all_f_params.append(f_params.numpy())
            all_k_params.append(k_params.numpy())
    
    # データ統合
    all_latents = np.vstack(all_latents)
    all_f_params = np.concatenate(all_f_params)
    all_k_params = np.concatenate(all_k_params)
    
    # 階層的クラスタリング
    hierarchical_clustering = HierarchicalClusteringAnalysis()
    hierarchical_clustering.fit(all_latents)
    
    # クラスタラベル取得
    cluster_labels = hierarchical_clustering.get_cluster_labels()
    
    # 包括的評価
    evaluator = ComprehensiveEvaluationMetrics()
    metrics = evaluator.calculate_all_metrics(
        all_latents, cluster_labels, all_f_params, all_k_params
    )
    
    # 結果表示
    evaluator.print_metrics()
    
    # 可視化
    visualize_phase4_results(all_latents, cluster_labels, all_f_params, all_k_params)
    
    return metrics

def visualize_phase4_results(latents, labels, f_params, k_params):
    """Phase 4結果の可視化"""
    
    # PCA
    pca = PCA(n_components=2)
    latents_pca = pca.fit_transform(latents)
    
    # t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(latents)//4))
    latents_tsne = tsne.fit_transform(latents)
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # PCA可視化
    scatter = axes[0, 0].scatter(latents_pca[:, 0], latents_pca[:, 1], c=labels, cmap='tab10', s=30)
    axes[0, 0].set_title('PCA Visualization')
    axes[0, 0].set_xlabel('PC1')
    axes[0, 0].set_ylabel('PC2')
    plt.colorbar(scatter, ax=axes[0, 0])
    
    # t-SNE可視化
    scatter = axes[0, 1].scatter(latents_tsne[:, 0], latents_tsne[:, 1], c=labels, cmap='tab10', s=30)
    axes[0, 1].set_title('t-SNE Visualization')
    axes[0, 1].set_xlabel('t-SNE 1')
    axes[0, 1].set_ylabel('t-SNE 2')
    plt.colorbar(scatter, ax=axes[0, 1])
    
    # f-k空間可視化
    scatter = axes[1, 0].scatter(f_params, k_params, c=labels, cmap='tab10', s=30)
    axes[1, 0].set_title('f-k Parameter Space')
    axes[1, 0].set_xlabel('f parameter')
    axes[1, 0].set_ylabel('k parameter')
    plt.colorbar(scatter, ax=axes[1, 0])
    
    # クラスタ分布
    unique_labels, counts = np.unique(labels, return_counts=True)
    axes[1, 1].bar(unique_labels, counts)
    axes[1, 1].set_title('Cluster Distribution')
    axes[1, 1].set_xlabel('Cluster')
    axes[1, 1].set_ylabel('Count')
    
    plt.tight_layout()
    plt.savefig('results/phase4_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()

if __name__ == "__main__":
    main() 


In [ ]:
# 環境セットアップ
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import imageio.v2 as imageio
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import time
import pickle

# GPU確認
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")
    device = torch.device('cuda')
    # GPU最適化設定
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
else:
    print("⚠️ GPU not available. Please enable GPU in Runtime > Change runtime type")
    device = torch.device('cpu')

print(f"🚀 Using device: {device}")

# パッケージインストール
try:
    import imageio
    import seaborn
    print("✅ All packages available")
except ImportError:
    print("📦 Installing required packages...")
    !pip install imageio scikit-learn seaborn
    import imageio
    import seaborn
    print("✅ Packages installed successfully")

# 追加パッケージ確認
try:
    import umap
    import hdbscan
    print("✅ UMAP/HDBSCAN available")
except Exception:
    print("📦 Installing umap-learn and hdbscan...")
    !pip install umap-learn hdbscan
    import umap
    import hdbscan
    print("✅ UMAP/HDBSCAN installed")


In [ ]:
# Google Drive接続とデータパス確認
from google.colab import drive

# Google Driveをマウント
drive.mount('/content/drive')

# データパス設定
GIF_FOLDER_PATH = '/content/drive/MyDrive/GrayScottML/gif'

# データ確認
if os.path.exists(GIF_FOLDER_PATH):
    gif_files = [f for f in os.listdir(GIF_FOLDER_PATH) if f.endswith('.gif')]
    gif_count = len(gif_files)
    print(f"✅ Google Drive connected successfully!")
    print(f"📁 Path: {GIF_FOLDER_PATH}")
    print(f"🎬 GIF files found: {gif_count}")
    
    if gif_count >= 1000:
        print("🎉 Ready for Phase 2 training with last 64 frames!")
    else:
        print(f"⚠️ Not enough files. Expected: 1500, Found: {gif_count}")
        print("Please upload more GIF files to Google Drive.")
else:
    print("❌ Google Drive path not found!")
    print(f"Expected path: {GIF_FOLDER_PATH}")
    print("Please ensure the following structure exists:")
    print("  MyDrive/")
    print("  └── GrayScottML/")
    print("      └── gif/")
    print("          ├── GrayScott-f0.0100-k0.0400-00.gif")
    print("          └── ... (1500 files)")
    
    # マイドライブの内容を表示
    mydrive_path = '/content/drive/MyDrive'
    if os.path.exists(mydrive_path):
        print(f"\n📂 Contents of MyDrive:")
        for item in sorted(os.listdir(mydrive_path))[:10]:
            print(f"   📁 {item}")
        print("   ... (showing first 10 items)")
    raise FileNotFoundError("Please set up the correct folder structure in Google Drive")


In [ ]:
# データセットクラス（後半64フレーム + Augmentation）
class GrayScottDatasetLast64Aug(Dataset):
    def __init__(self, gif_folder, fixed_frames=64, target_size=(64, 64), max_samples=None, augment=True, max_shift_ratio=0.1):
        self.gif_folder = gif_folder
        self.fixed_frames = fixed_frames
        self.target_size = target_size
        self.max_samples = max_samples
        self.augment = augment
        self.max_shift_ratio = max_shift_ratio
        
        self.gif_files = []
        self.f_values = []
        self.k_values = []
        self.tensors = []
        
        self._load_data()
    
    def _parse_filename(self, filename):
        pattern = r"GrayScott-f([0-9.]+)-k([0-9.]+)-\d+\.gif"
        m = re.match(pattern, filename)
        if m:
            return float(m.group(1)), float(m.group(2))
        return None, None
    
    def _load_gif_as_tensor(self, gif_path):
        try:
            gif = imageio.mimread(gif_path)
            total_frames = len(gif)
            if total_frames >= self.fixed_frames:
                start_idx = total_frames - self.fixed_frames
                selected_frames = gif[start_idx:]
            else:
                selected_frames = gif
            frames = []
            for frame in selected_frames:
                if len(frame.shape) == 3:
                    frame = np.mean(frame, axis=2)
                pil_frame = Image.fromarray(frame.astype(np.uint8))
                pil_frame = pil_frame.resize(self.target_size)
                frame_array = np.array(pil_frame) / 255.0
                frames.append(frame_array)
            while len(frames) < self.fixed_frames:
                frames.append(frames[-1] if frames else np.zeros(self.target_size))
            frames = frames[:self.fixed_frames]
            tensor = torch.FloatTensor(np.array(frames))  # (T, H, W)
            return tensor.unsqueeze(0)  # (1, T, H, W)
        except Exception as e:
            print(f"Error loading {gif_path}: {e}")
            return None
    
    def _load_data(self):
        gif_files = [f for f in os.listdir(self.gif_folder) if f.endswith('.gif')]
        if self.max_samples:
            gif_files = gif_files[:self.max_samples]
        print(f"Loading {len(gif_files)} GIF files (last 64 frames each) with augmentation={self.augment}...")
        for i, filename in enumerate(gif_files):
            if i % 100 == 0:
                print(f"Progress: {i+1}/{len(gif_files)} ({(i+1)/len(gif_files)*100:.1f}%)")
            f_val, k_val = self._parse_filename(filename)
            if f_val is None or k_val is None:
                continue
            gif_path = os.path.join(self.gif_folder, filename)
            tensor = self._load_gif_as_tensor(gif_path)
            if tensor is not None:
                self.gif_files.append(filename)
                self.f_values.append(f_val)
                self.k_values.append(k_val)
                self.tensors.append(tensor)
        print(f"✅ Successfully loaded {len(self.tensors)} samples")
    
    def __len__(self):
        return len(self.tensors)
    
    def _random_flip(self, x):
        # x: (1, T, H, W)
        if np.random.rand() < 0.5:
            x = torch.flip(x, dims=[-1])  # horizontal
        if np.random.rand() < 0.5:
            x = torch.flip(x, dims=[-2])  # vertical
        return x
    
    def _random_rotate_90(self, x):
        k = np.random.choice([0,1,2,3])
        if k:
            x = torch.rot90(x, k=k, dims=(-2, -1))
        return x
    
    def _random_translate(self, x):
        # zero-padded translation by up to max_shift_ratio of size
        _, _, H, W = x.shape
        max_dx = int(W * self.max_shift_ratio)
        max_dy = int(H * self.max_shift_ratio)
        if max_dx == 0 and max_dy == 0:
            return x
        dx = int(np.random.randint(-max_dx, max_dx+1))
        dy = int(np.random.randint(-max_dy, max_dy+1))
        if dx == 0 and dy == 0:
            return x
        pad_left = max(dx, 0)
        pad_right = max(-dx, 0)
        pad_top = max(dy, 0)
        pad_bottom = max(-dy, 0)
        x_padded = F.pad(x, (pad_left, pad_right, pad_top, pad_bottom), mode='constant', value=0.0)
        x_cropped = x_padded[:, :, pad_bottom:pad_bottom+H, pad_right:pad_right+W]
        return x_cropped
    
    def _apply_augmentation(self, x):
        x = self._random_flip(x)
        x = self._random_rotate_90(x)
        x = self._random_translate(x)
        return x
    
    def __getitem__(self, idx):
        sample = self.tensors[idx].clone()  # (1, T, H, W)
        if self.augment:
            sample = self._apply_augmentation(sample)
        return {
            'tensor': sample,
            'f_value': self.f_values[idx],
            'k_value': self.k_values[idx],
            'filename': self.gif_files[idx]
        }


In [ ]:
# データセットクラス（後半64フレーム専用）
class GrayScottDatasetLast64(Dataset):
    def __init__(self, gif_folder, fixed_frames=64, target_size=(64, 64), max_samples=None):
        self.gif_folder = gif_folder
        self.fixed_frames = fixed_frames
        self.target_size = target_size
        self.max_samples = max_samples
        
        self.gif_files = []
        self.f_values = []
        self.k_values = []
        self.tensors = []
        
        self._load_data()
    
    def _parse_filename(self, filename):
        pattern = r'GrayScott-f([0-9.]+)-k([0-9.]+)-\d+\.gif'
        match = re.match(pattern, filename)
        if match:
            return float(match.group(1)), float(match.group(2))
        return None, None
    
    def _load_gif_as_tensor(self, gif_path):
        try:
            gif = imageio.mimread(gif_path)
            total_frames = len(gif)
            
            # 後半64フレームを抽出
            if total_frames >= self.fixed_frames:
                # 後半64フレームを取得
                start_idx = total_frames - self.fixed_frames
                selected_frames = gif[start_idx:]
            else:
                # フレーム数が64未満の場合は全フレームを使用
                selected_frames = gif
                print(f"Warning: Only {total_frames} frames available, using all frames")
            
            frames = []
            for frame in selected_frames:
                if len(frame.shape) == 3:
                    frame = np.mean(frame, axis=2)
                
                pil_frame = Image.fromarray(frame.astype(np.uint8))
                pil_frame = pil_frame.resize(self.target_size)
                frame_array = np.array(pil_frame) / 255.0
                frames.append(frame_array)
            
            # 64フレームになるようにパディング（必要に応じて）
            while len(frames) < self.fixed_frames:
                frames.append(frames[-1] if frames else np.zeros(self.target_size))
            
            # 正確に64フレームにする
            frames = frames[:self.fixed_frames]
            
            tensor = torch.FloatTensor(np.array(frames))
            return tensor.unsqueeze(0)  # チャンネル次元を追加
            
        except Exception as e:
            print(f"Error loading {gif_path}: {e}")
            return None
    
    def _load_data(self):
        gif_files = [f for f in os.listdir(self.gif_folder) if f.endswith('.gif')]
        
        if self.max_samples:
            gif_files = gif_files[:self.max_samples]
        
        print(f"Loading {len(gif_files)} GIF files (last 64 frames each)...")
        
        for i, filename in enumerate(gif_files):
            if i % 100 == 0:
                print(f"Progress: {i+1}/{len(gif_files)} ({(i+1)/len(gif_files)*100:.1f}%)")
            
            f_val, k_val = self._parse_filename(filename)
            if f_val is None or k_val is None:
                continue
            
            gif_path = os.path.join(self.gif_folder, filename)
            tensor = self._load_gif_as_tensor(gif_path)
            
            if tensor is not None:
                self.gif_files.append(filename)
                self.f_values.append(f_val)
                self.k_values.append(k_val)
                self.tensors.append(tensor)
        
        print(f"✅ Successfully loaded {len(self.tensors)} samples with last 64 frames each")
    
    def __len__(self):
        return len(self.tensors)
    
    def __getitem__(self, idx):
        return {
            'tensor': self.tensors[idx],
            'f_value': self.f_values[idx],
            'k_value': self.k_values[idx],
            'filename': self.gif_files[idx]
        }


In [ ]:
# 注意機構モジュール
class SpatioTemporalAttention(nn.Module):
    def __init__(self, channels):
        super().__init__()
        
        # 空間注意
        self.spatial_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d((None, 1, 1)),
            nn.Conv3d(channels, channels//4, 1),
            nn.ReLU(inplace=True),
            nn.Conv3d(channels//4, channels, 1),
            nn.Sigmoid()
        )
        
        # 時間注意
        self.temporal_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d((1, None, None)),
            nn.Conv3d(channels, channels//4, 1),
            nn.ReLU(inplace=True),
            nn.Conv3d(channels//4, channels, 1),
            nn.Sigmoid()
        )
        
        # チャンネル注意
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d(1),
            nn.Conv3d(channels, channels//4, 1),
            nn.ReLU(inplace=True),
            nn.Conv3d(channels//4, channels, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        identity = x
        
        # 3つの注意機構を適用
        x = x * self.spatial_attention(x)
        x = x * self.temporal_attention(x)
        x = x * self.channel_attention(x)
        
        return x + identity * 0.1

# 残差注意ブロック
class ResidualAttentionBlock3D(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv3d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm3d(out_channels, momentum=0.1)
        self.conv2 = nn.Conv3d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm3d(out_channels, momentum=0.1)
        
        self.attention = SpatioTemporalAttention(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm3d(out_channels, momentum=0.1)
            )
        
        self.dropout = nn.Dropout3d(0.1)
    
    def forward(self, x):
        identity = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        out = self.attention(out)
        
        out += self.shortcut(identity)
        return F.relu(out)


## 🚀 Step 3: Phase 2 学習実行（64フレーム対応）


In [ ]:
# Phase 4 architecture
# 利用するモデル: src/gray_scott_autoencoder_phase3.py の Conv3DAutoencoderPhase3 を使用します。
# using embedded Conv3DAutoencoderPhase4 (import removed)


In [ ]:
# Phase 2 メインモデル（64フレーム対応）
class Conv3DAutoencoderPhase4Last64(nn.Module):
    def __init__(self, input_channels=1, fixed_frames=64, target_size=(64, 64), latent_dim=256):
        super().__init__()
        
        self.latent_dim = latent_dim
        self.fixed_frames = fixed_frames
        self.target_size = target_size
        
        # 初期畳み込み（64フレーム対応）
        self.initial_conv = nn.Sequential(
            nn.Conv3d(input_channels, 32, (5, 7, 7), (2, 2, 2), (2, 3, 3)),
            nn.BatchNorm3d(32, momentum=0.1),
            nn.ReLU(inplace=True),
            nn.Dropout3d(0.05)
        )
        
        # 残差注意ブロック群（64フレーム用に調整）
        self.res_block1 = ResidualAttentionBlock3D(32, 64, stride=(2, 2, 2))
        self.res_block2 = ResidualAttentionBlock3D(64, 64)
        self.res_block3 = ResidualAttentionBlock3D(64, 128, stride=(2, 2, 2))
        self.res_block4 = ResidualAttentionBlock3D(128, 128)
        self.res_block5 = ResidualAttentionBlock3D(128, 256, stride=(2, 2, 2))
        self.res_block6 = ResidualAttentionBlock3D(256, 256)
        
        # グローバルプーリング
        self.global_pool = nn.AdaptiveAvgPool3d((2, 2, 2))
        self.dropout_before_latent = nn.Dropout3d(0.3)
        
        # 潜在空間射影
        self.to_latent = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 2 * 2 * 2, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, latent_dim),
            nn.BatchNorm1d(latent_dim)
        )
        
        # 復元
        self.from_latent = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, 256 * 2 * 2 * 2),
            nn.ReLU(inplace=True)
        )
        
        # デコーダー（64フレーム復元用）
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(256, 128, (4, 4, 4), (2, 2, 2), (1, 1, 1)),
            nn.BatchNorm3d(128, momentum=0.1),
            nn.ReLU(inplace=True),
            nn.Dropout3d(0.2),
            
            nn.ConvTranspose3d(128, 64, (4, 4, 4), (2, 2, 2), (1, 1, 1)),
            nn.BatchNorm3d(64, momentum=0.1),
            nn.ReLU(inplace=True),
            nn.Dropout3d(0.15),
            
            nn.ConvTranspose3d(64, 32, (4, 4, 4), (2, 2, 2), (1, 1, 1)),
            nn.BatchNorm3d(32, momentum=0.1),
            nn.ReLU(inplace=True),
            nn.Dropout3d(0.1),
            
            nn.ConvTranspose3d(32, input_channels, (6, 7, 7), (2, 2, 2), (2, 3, 3)),
            nn.Sigmoid()
        )
    
    def encode(self, x):
        x = self.initial_conv(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = self.res_block5(x)
        x = self.res_block6(x)
        x = self.global_pool(x)
        x = self.dropout_before_latent(x)
        return self.to_latent(x)
    
    def decode(self, latent):
        x = self.from_latent(latent)
        x = x.view(-1, 256, 2, 2, 2)
        x = self.decoder(x)
        target_h, target_w = self.target_size
        return F.interpolate(x, size=(self.fixed_frames, target_h, target_w), 
                           mode='trilinear', align_corners=False)
    
    def forward(self, x):
        latent = self.encode(x)
        reconstructed = self.decode(latent)
        return reconstructed, latent


In [ ]:
# Phase 4 学習・評価実行（64フレーム版）
def run_phase4_training_last64():
    # パラメータ設定
    fixed_frames = 64  # 後半64フレーム
    target_size = (64, 64)
    latent_dim = 512
    num_epochs = 50
    batch_size = 6 if torch.cuda.is_available() else 3  # 64フレームなので少し小さく
    learning_rate = 1e-3
    weight_decay = 1e-4
    n_clusters = 5
    
    print("🔄 Creating dataset (last 64 frames, with augmentation)...")
    dataset = GrayScottDatasetLast64Aug(GIF_FOLDER_PATH, fixed_frames, target_size, augment=True, max_samples=max_samples)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                          num_workers=2, pin_memory=True)
    
    print(f"📊 Dataset: {len(dataset)} samples, Batch size: {batch_size}, Frames: {fixed_frames}")
    
    print("🧠 Creating Phase 4 model (64 frames)...")
    model = Conv3DAutoencoderPhase4(latent_dim=latent_dim, 
                                         fixed_frames=fixed_frames, 
                                         target_size=target_size).to(device)
    
    print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # 訓練
    print("🎯 Starting training with last 64 frames...")
    model.train()
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)
    
    losses = []
    start_time = time.time()
    
    print(f"🚀 Phase 4 GPU Training: ResNet + Attention (Last 64 Frames)")
    print("=" * 80)
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        epoch_loss = 0.0
        
        for batch_idx, batch in enumerate(dataloader):
            tensors = batch['tensor'].to(device, non_blocking=True)
            
            optimizer.zero_grad()
            reconstructed, latent = model(tensors)
            loss = criterion(reconstructed, tensors)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()
            epoch_loss += loss.item()
        
        scheduler.step()
        
        avg_loss = epoch_loss / len(dataloader)
        losses.append(avg_loss)
        current_lr = scheduler.get_last_lr()[0]
        epoch_time = time.time() - epoch_start_time
        
        # 進捗表示
        progress = ((epoch + 1) / num_epochs) * 100
        if (epoch + 1) % 5 == 0 or epoch < 5:
            print(f'Epoch [{epoch+1:2d}/{num_epochs}] '
                  f'({progress:5.1f}%) | '
                  f'Loss: {avg_loss:.6f} | '
                  f'LR: {current_lr:.2e} | '
                  f'Time: {epoch_time:.1f}s')
    
    total_time = time.time() - start_time
    print("=" * 80)
    print(f"🎉 Training completed in {total_time/60:.1f} minutes")
    
    # 学習曲線表示
    plt.figure(figsize=(10, 6))
    plt.plot(losses, linewidth=2, color='purple')
    plt.title('Phase 4: GPU Training Loss (ResNet + Attention, Last 64 Frames)', fontsize=14)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 潜在ベクトル抽出
    print("🔍 Extracting latent vectors...")
    # Rebuild dataset without augmentation for evaluation/feature extraction
    dataset_eval = GrayScottDatasetLast64Aug(GIF_FOLDER_PATH, fixed_frames, target_size, augment=False, max_samples=max_samples)
    dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    model.eval()
    latent_vectors = []
    f_values = []
    k_values = []
    
    with torch.no_grad():
        for batch in dataloader_eval:
            tensors = batch['tensor'].to(device)
            _, latent = model(tensors)
            latent_vectors.append(latent.cpu().numpy())
            f_values.extend(batch['f_value'].numpy())
            k_values.extend(batch['k_value'].numpy())
    
    latent_vectors = np.vstack(latent_vectors)
    f_values = np.array(f_values)
    k_values = np.array(k_values)
    
    # クラスタリング
    print("�� Performing clustering with multiple strategies...")
from sklearn.decomposition import PCA as _PCA
from sklearn.preprocessing import StandardScaler as _SS
from sklearn.cluster import KMeans as _KMeans
# Strategy A: PCA(whiten=True) -> KMeans(n_init=50)
Xa = _PCA(n_components=min(64, latent_vectors.shape[1]), whiten=True, random_state=42).fit_transform(latent_vectors)
labels_a = _KMeans(n_clusters=n_clusters, n_init=50, random_state=42).fit_predict(Xa)
sil_a = silhouette_score(latent_vectors, labels_a)
print(f"[A] PCA(whiten)+KMeans(n_init=50): Silhouette={sil_a:.4f}")

# Strategy B: L2-normalize -> KMeans(n_init=50)  (cosine近似)
Xb = latent_vectors / (np.linalg.norm(latent_vectors, axis=1, keepdims=True) + 1e-8)
labels_b = _KMeans(n_clusters=n_clusters, n_init=50, random_state=42).fit_predict(Xb)
sil_b = silhouette_score(latent_vectors, labels_b)
print(f"[B] L2-norm+KMeans(n_init=50): Silhouette={sil_b:.4f}")

# Strategy C: UMAP -> HDBSCAN
try:
    import umap
    import hdbscan
    U = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=10, random_state=42).fit_transform(latent_vectors)
    labels_c = hdbscan.HDBSCAN(min_cluster_size=20).fit_predict(U)
    # -1 はノイズ
    valid = labels_c != -1
    if valid.any() and len(set(labels_c[valid]))>1:
        sil_c = silhouette_score(latent_vectors[valid], labels_c[valid])
        print(f"[C] UMAP+HDBSCAN: clusters={len(set(labels_c[valid]))}, noise={(~valid).sum()}, Silhouette(valid)={sil_c:.4f}")
    else:
        print(f"[C] UMAP+HDBSCAN: clusters={len(set(labels_c)) if len(set(labels_c))>1 else 0}, noise={(labels_c==-1).sum()} (silhouette N/A)")
except Exception as e:
    print(f"[C] UMAP+HDBSCAN unavailable: {e}")

# 既定の出力を[A]に設定
cluster_labels = labels_a
# Phase 1との比較
phase1_score = 0.565
improvement = ((silhouette_avg - phase1_score) / phase1_score) * 100

print(f"📈 Performance Comparison:")
print(f"   Phase 1 (30 frames): {phase1_score:.4f}")
print(f"   Phase 4 (last 64 frames): {silhouette_avg:.4f}")
print(f"   Improvement: {improvement:+.1f}%")

if improvement >= 15:
print("🎉 Phase 4 目標達成！ (15%以上の向上) - Last 64 frames strategy successful!")
else:
print(f"⚠️  Phase 4 目標未達 ({improvement:.1f}% < 15%) - Consider further optimization")

return {
'model': model,
'losses': losses,
'silhouette_score': silhouette_avg,
'calinski_score': calinski_score,
'davies_bouldin': davies_bouldin,
'latent_vectors': latent_vectors,
'cluster_labels': cluster_labels,
'f_values': f_values,
'k_values': k_values,
'improvement': improvement,
'frames_used': 'last_64'
}

# 実行
print("🚀 Starting Phase 4 training with last 64 frames...")
results = run_phase4_training_last64()


In [ ]:
# Ensure latent_vectors are available if only results exist
if 'latent_vectors' not in globals():
    if 'results' in globals() and isinstance(results, dict) and 'latent_vectors' in results:
        latent_vectors = results['latent_vectors']
        f_values = results.get('f_values', None)
        k_values = results.get('k_values', None)
        cluster_labels = results.get('cluster_labels', None)
        print('Recovered latent_vectors from results dict:', latent_vectors.shape)
    else:
        print('latent_vectors not found. Run training to produce results and latent vectors.')


## 📥 Step 4: 結果保存・ダウンロード

In [ ]:
# 結果保存とダウンロード（Last 64 frames版）
import pickle
from google.colab import files

# 結果をGoogle Driveに保存
results_path = '/content/drive/MyDrive/GrayScottML/phase4_results_last64_gpu.pkl'
model_path = '/content/drive/MyDrive/GrayScottML/phase4_model_last64_gpu.pth'

# 結果保存
with open(results_path, 'wb') as f:
    # モデルは除いて保存（サイズ削減）
    save_results = {k: v for k, v in results.items() if k != 'model'}
    pickle.dump(save_results, f)

# モデル保存
torch.save(results['model'].state_dict(), model_path)

print(f"💾 Results saved to Google Drive:")
print(f"   📊 Results: {results_path}")
print(f"   🧠 Model: {model_path}")

# ローカルにもダウンロード用ファイル作成
local_results_path = 'phase4_results_last64_gpu.pkl'
local_model_path = 'phase4_model_last64_gpu.pth'

with open(local_results_path, 'wb') as f:
    save_results = {k: v for k, v in results.items() if k != 'model'}
    pickle.dump(save_results, f)

torch.save(results['model'].state_dict(), local_model_path)

print("\n📥 Downloading files...")
files.download(local_results_path)
files.download(local_model_path)

print("✅ Download completed!")
print(f"🎯 Final Results (Last 64 Frames):")
print(f"   ⭐ Silhouette Score: {results['silhouette_score']:.4f}")
print(f"   📈 Improvement: {results['improvement']:+.1f}%")
print(f"   🎬 Strategy: Using last 64 frames for more stable patterns")
print(f"   🏆 Target: {'✅ ACHIEVED' if results['improvement'] >= 15 else '❌ NOT ACHIEVED'}")


## 📊 Step 5: 可視化・分析（Optional）


In [ ]:
# Configure Google Drive save directory for figures
import os
SAVE_DIR = os.environ.get('GSML_SAVE_DIR', '/content/drive/MyDrive/GrayScottML/results')
os.makedirs(SAVE_DIR, exist_ok=True)
print('Figures will be saved to:', SAVE_DIR)


In [ ]:
# 結果の詳細可視化（Last 64 frames）
def visualize_results_last64(results):
    latent_vectors = results['latent_vectors']
    cluster_labels = results['cluster_labels']
    f_values = results['f_values']
    k_values = results['k_values']
    
    # PCAとt-SNEによる次元削減
    print("🔍 Performing dimensionality reduction...")
    pca = PCA(n_components=2, random_state=42)
    pca_result = pca.fit_transform(latent_vectors)
    
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    tsne_result = tsne.fit_transform(latent_vectors)
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Phase 2 Results: Phase 4 (Last 64 Frames) Analysis', fontsize=16, fontweight='bold')
    
    # f-k空間でのクラスタリング結果
    scatter1 = axes[0, 0].scatter(f_values, k_values, c=cluster_labels, cmap='tab10', alpha=0.7, s=20)
    axes[0, 0].set_xlabel('f parameter')
    axes[0, 0].set_ylabel('k parameter')
    axes[0, 0].set_title('Clustering Results in f-k Parameter Space')
    axes[0, 0].invert_yaxis()
    plt.colorbar(scatter1, ax=axes[0, 0], label='Cluster ID')
    
    # PCA結果
    scatter2 = axes[0, 1].scatter(pca_result[:, 0], pca_result[:, 1], c=cluster_labels, cmap='tab10', alpha=0.7, s=20)
    axes[0, 1].set_title(f'PCA of Latent Space (Phase 4 (Last 64 Frames))')
    axes[0, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    axes[0, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    plt.colorbar(scatter2, ax=axes[0, 1], label='Cluster ID')
    
    # t-SNE結果
    scatter3 = axes[1, 0].scatter(tsne_result[:, 0], tsne_result[:, 1], c=cluster_labels, cmap='tab10', alpha=0.7, s=20)
    axes[1, 0].set_title('t-SNE of Latent Space (Phase 4 (Last 64 Frames))')
    plt.colorbar(scatter3, ax=axes[1, 0], label='Cluster ID')
    
    # クラスター統計
    axes[1, 1].axis('off')
    n_clusters = len(np.unique(cluster_labels))
    stats_text = f"Phase 4 (Last 64 Frames) Analysis:\\n\\n"
    stats_text += f"Silhouette Score: {results['silhouette_score']:.4f}\\n"
    stats_text += f"Improvement: {results['improvement']:+.1f}%\\n\\n"
    
    for i in range(n_clusters):
        mask = cluster_labels == i
        count = np.sum(mask)
        f_mean = f_values[mask].mean()
        k_mean = k_values[mask].mean()
        stats_text += f"Cluster {i}: {count} samples\\n"
        stats_text += f"  f_avg: {f_mean:.4f}\\n"
        stats_text += f"  k_avg: {k_mean:.4f}\\n\\n"
    
    axes[1, 1].text(0.1, 0.9, stats_text, transform=axes[1, 1].transAxes, 
                    fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, 'phase4_clustering_overview.png'), dpi=200, bbox_inches='tight'); plt.close()
    
    print("📊 Visualization completed!")
    print(f"🎬 Strategy: Last 64 frames captured more stable, mature patterns")
    print(f"⭐ Final Score: {results['silhouette_score']:.4f}")

# 可視化実行
print("🎨 Creating visualizations...")
visualize_results_last64(results)
